In [1]:
import apache_beam as beam # type: ignore
import json

# Ler arquivo texto, separar e mostrar

In [5]:

p1 = beam.Pipeline()

voos = (
p1
| "Importar Dados" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines=1)
| "Separar por vírgula" >> beam.Map(lambda record: record.split(','))
| "Mostrar resultados" >> beam.Map(print)
)

p1.run()

['2019-04-27', '19805', '2', 'LAX', 'JFK', '944', '14', '1736', '-29', '269', '2475', '2']
['2019-04-27', '19805', '3', 'JFK', 'LAX', '1224', '-6', '1614', '39', '371', '2475', '3']
['2019-04-27', '19805', '4', 'LAX', 'JFK', '1240', '25', '2028', '-27', '264', '2475', '4']
['2019-04-27', '19805', '5', 'DFW', 'HNL', '1300', '-5', '1650', '15', '510', '3784', '5']
['2019-04-27', '19805', '6', 'OGG', 'DFW', '1901', '126', '640', '95', '385', '3711', '6']
['2019-04-27', '19805', '7', 'DFW', 'OGG', '1410', '125', '1743', '138', '497', '3711', '7']
['2019-04-27', '19805', '8', 'HNL', 'DFW', '1659', '4', '458', '-22', '398', '3784', '8']
['2019-04-27', '19805', '9', 'JFK', 'LAX', '648', '-7', '1029', '19', '365', '2475', '9']
['2019-04-27', '19805', '10', 'LAX', 'JFK', '2156', '21', '556', '1', '265', '2475', '10']
['2019-04-27', '19805', '12', 'LAX', 'JFK', '1113', '-2', '1910', '-40', '267', '2475', '11']
['2019-04-27', '19805', '14', 'OGG', 'LAX', '2235', '5', '618', '-17', '270', '2486', 

# Criar objetos (Tupla e Lista)

In [6]:
p1 = beam.Pipeline()

p1 | "Tupla" >> beam.Create( [ ("Felipe", 28), ("Andressa", 42) ] ) | "Print Tupla" >> beam.Map(print)
p1 | "Lista" >> beam.Create( [ 1, 2, 3 ] ) | "Print Lista" >> beam.Map(print)

p1.run()

1
2
3
('Felipe', 28)
('Andressa', 42)


# Gravar objeto ou arquivos

In [7]:
gravarObjeto = beam.Pipeline()

voos = (
    gravarObjeto
    | "Import Data" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines = 1) 
    | "Split" >> beam.Map(lambda record: record.split(','))
    # | "Show results" >> beam.Map(print)
    | "Write data" >> beam.io.WriteToText("voos.txt")
)

gravarObjeto.run()

# Command Map e Flat Map

In [28]:
#Flat map example
p1 = beam.Pipeline()

Colletion = (
    p1
    |beam.io.ReadFromText('poema.txt')
    |beam.FlatMap(lambda record: record.split(' '))
    |beam.io.WriteToText('results.txt')
)

p1.run()

# Filter list and lambda

In [29]:

p1 = beam.Pipeline()

#lambda
voos = (
p1
| "Import Data" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines=1)
| "Split" >> beam.Map(lambda record: record.split(','))
| "Get Los Angels flights" >> beam.Filter(lambda record: record[3] == "LAX")
| "Show results" >> beam.Map(print)
)

p1.run()

['2019-04-27', '19805', '2', 'LAX', 'JFK', '944', '14', '1736', '-29', '269', '2475', '2']
['2019-04-27', '19805', '4', 'LAX', 'JFK', '1240', '25', '2028', '-27', '264', '2475', '4']
['2019-04-27', '19805', '10', 'LAX', 'JFK', '2156', '21', '556', '1', '265', '2475', '10']
['2019-04-27', '19805', '12', 'LAX', 'JFK', '1113', '-2', '1910', '-40', '267', '2475', '11']
['2019-04-27', '19805', '22', 'LAX', 'JFK', '1458', '-2', '2336', '11', '272', '2475', '20']


In [31]:
words = ['quatro', 'um']

def findWord( i ) :
    if i in words:
        return True

p1 = beam.Pipeline()
Collection = (
    p1
    |beam.io.ReadFromText('poema.txt')
    |beam.FlatMap(lambda record: record.split(' '))
    |beam.Filter(findWord)
    |beam.io.WriteToText('results_filtered.txt')
)

p1.run()

# Flatten
### Union with same data type in one Pcolletion

In [33]:
p = beam.Pipeline()

negros = ('Adão', 'Jesus', 'Mike')
brancos = ('Tulio', 'Mary', 'Joca')
indios = ('Vic', 'Marta', 'Tom')

negros_pc = p | "Criando Pcolletion negros" >> beam.Create(negros)
brancos_pc = p | "Criando Pcolletion brancos" >> beam.Create(brancos)
indios_pc = p | "Criando Pcolletion indios" >> beam.Create(indios)

pessoas = ((negros_pc, brancos_pc, indios_pc) | beam.Flatten()) | beam.Map(print)
p.run()

Vic
Marta
Tom
Tulio
Mary
Joca
Adão
Jesus
Mike


# CombinePerKey

In [35]:
p1 = beam.Pipeline()

Tempo_Atrasos = (
    p1
    | "Import data" >> beam.io.ReadFromText('voos_sample.csv')
    | "Split" >> beam.Map(lambda record: record.split(','))
    | "LA Flights" >> beam.Filter(lambda record: int(record[8]) > 0 )
    | "Create Pair" >> beam.Map(lambda record: (record[4], int(record[8]) ) )
    | "Sum by key" >> beam.CombinePerKey(sum)
    | "Show results" >> beam.Map(print)
)

p1.run()

('LAX', 94)
('HNL', 15)
('DFW', 95)
('OGG', 138)
('JFK', 220)


# Combiners.Count.PerKey

In [8]:
p1 = beam.Pipeline()

Qtd_Atrasos = (
    p1
    | "Import data" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines=1)
    | "Split" >> beam.Map(lambda record: record.split(','))
    | "LA Flights" >> beam.Filter(lambda record: int(record[8]) > 0 )
    | "Create Pair" >> beam.Map(lambda record: (record[4], int(record[8]) ) )
    | "Count by key" >> beam.combiners.Count.PerKey()
    | "Show results" >> beam.Map(print)
)

p1.run()

('LAX', 4)
('HNL', 1)
('DFW', 1)
('OGG', 1)
('JFK', 4)


# CoGroupByKey

In [10]:
p1 = beam.Pipeline()

Tempo_atrasos = (
    p1
    | "Delay Import date" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines= 1)
    | "Delay Split" >> beam.Map(lambda record: record.split(',') )
    | "Delay Get Flights" >> beam.Filter(lambda record: int(record[8]) > 0 )
    | "Delay create pair" >> beam.Map(lambda record: (record[4], int(record[8]) ) )
    | "Delay Sum by key" >> beam.CombinePerKey(sum)
    # | "Show results" >> beam.Map(print)
)

Qtd_Atrasos = (
    p1
    | "QTD Import Data" >> beam.io.ReadFromText("voos_sample.csv", skip_header_lines= 1)
    | "QTD Split" >> beam.Map(lambda record: record.split(',') )
    | "QTD get delay flights" >> beam.Filter(lambda record: int(record[8]) > 0 )
    | "QTD create pair" >> beam.Map(lambda record: (record[4], int(record[8]) ) )
    | "QTD count by key" >> beam.combiners.Count.PerKey()
    # | "Show qtd results" >> beam.Map(print)
)

Delay_tables = (
    {'QTD_Delay' : Qtd_Atrasos, 'Delay_Time': Tempo_atrasos}
    | "Group BY" >> beam.CoGroupByKey()
    | beam.Map(print)
)

p1.run()

('LAX', {'QTD_Delay': [4], 'Delay_Time': [92]})
('HNL', {'QTD_Delay': [1], 'Delay_Time': [15]})
('DFW', {'QTD_Delay': [1], 'Delay_Time': [95]})
('OGG', {'QTD_Delay': [1], 'Delay_Time': [138]})
('JFK', {'QTD_Delay': [4], 'Delay_Time': [220]})


# ParDo

## declara funções com beam.DoFN

In [16]:
p1 = beam.Pipeline()

class filtro(beam.DoFn):
    def process(self, record):
        if int(record[8]) > 0:
            return [record]


Tempo_Atraso = (
    p1
    | "Import Data" >> beam.io.ReadFromText(r"voos_sample.csv", skip_header_lines=1)
    | "Split data" >> beam.Map(lambda record: record.split(','))
    | "Delay Flights" >> beam.ParDo(filtro())
    | "Create pair" >> beam.Map(lambda record: (record[4], int(record[8]) ) )
    | "Sum by key" >> beam.CombinePerKey(sum)
)


Qtd_Atrasos = (
    p1
    | "QTD Import Data" >> beam.io.ReadFromText(r"voos_sample.csv", skip_header_lines= 1)
    | "QTD Split data" >> beam.Map(lambda record: record.split(','))
    | "QTD Delay Flights" >> beam.ParDo(filtro())
    | "QTD Create pair" >> beam.Map(lambda record: (record[4], int(record[8]) ) )
    | "QTD Count by key" >> beam.combiners.Count.PerKey()
)

Tabela_delay = (
    {'QTD_atrasos' : Qtd_Atrasos, 'Tempo_atrasos': Tempo_Atraso}
    | "QTD Group by" >> beam.CoGroupByKey()
    | beam.Map(print)
)

p1.run()

('LAX', {'QTD_atrasos': [4], 'Tempo_atrasos': [92]})
('HNL', {'QTD_atrasos': [1], 'Tempo_atrasos': [15]})
('DFW', {'QTD_atrasos': [1], 'Tempo_atrasos': [95]})
('OGG', {'QTD_atrasos': [1], 'Tempo_atrasos': [138]})
('JFK', {'QTD_atrasos': [4], 'Tempo_atrasos': [220]})
